In [1]:
import undetected_chromedriver as uc
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.action_chains import ActionChains
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
from lxml import html
import joblib
import datetime

def remove_duplicates(lst):
    x=[]
    for i in lst:
        if i not in x:
            x.append(i)
    return x
def assign(var,xpath_str):
    try:
        var=tree.xpath(xpath_str)[0].strip()
        if var.isdigit():
            var=int(var)
    except:
        var=''
    return var

In [2]:
username='NRehmtulla'
password='Asc12345!'

In [3]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--incognito")

driver = uc.Chrome(executable_path=ChromeDriverManager().install(),use_subprocess=True,options=chrome_options)
driver.delete_all_cookies()
driver.get("https://apps.semadata.org/")
try:
    driver.minimize_window()
except:
    pass
try:
    driver.maximize_window()
except:
    pass
driver.implicitly_wait(30)
driver.find_element(By.XPATH,"//input[@placeholder='Username']").send_keys(username)
driver.find_element(By.XPATH,"//input[@placeholder='Password']").send_keys(password)
driver.find_element(By.ID,'login-button').click()
driver.find_element(By.XPATH,'//li[@data-name="Product Lookup"]/a').click()
driver.set_script_timeout(1000)


In [4]:
try:
    all_data=joblib.load('all_brands_data.sav')
    brands_info=joblib.load('brands_info.sav')
except:
    all_data=[]
print(len(all_data))

191657


In [5]:
if len(all_data)==0:
    brands_info=[]
# select_brand=Select(driver.find_element(By.ID,'receiver-brands'))
    for i in range(1,len(Select(driver.find_element(By.ID,'receiver-brands')).options)):
        brand_id=driver.find_element(By.ID,'receiver-brands').find_elements(By.TAG_NAME,'option')[i].get_attribute('value')
        Select(driver.find_element(By.ID,'receiver-brands')).select_by_index(i)
        brand_name=driver.find_element(By.ID,'receiver-brands').find_elements(By.TAG_NAME,'option')[i].text
        temp_lst=driver.execute_script('''
            var datas;
            await fetch("https://apps.semadata.org/Receiver/GetProducts", {
              "headers": {
                "accept": "application/json, text/javascript, */*; q=0.01",
                "accept-language": "en-GB,en-US;q=0.9,en;q=0.8",
                "content-type": "application/x-www-form-urlencoded; charset=UTF-8",
                "sec-fetch-dest": "empty",
                "sec-fetch-mode": "cors",
                "sec-fetch-site": "same-origin",
                "x-requested-with": "XMLHttpRequest"
              },
              "referrer": "https://apps.semadata.org/",
              "referrerPolicy": "strict-origin-when-cross-origin",
              "body": "draw=2&columns%5B0%5D%5Bdata%5D=PartNumber&columns%5B0%5D%5Bname%5D=&columns%5B0%5D%5Bsearchable%5D=true&columns%5B0%5D%5Borderable%5D=true&columns%5B0%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B0%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B1%5D%5Bdata%5D=CategoryName&columns%5B1%5D%5Bname%5D=&columns%5B1%5D%5Bsearchable%5D=true&columns%5B1%5D%5Borderable%5D=true&columns%5B1%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B1%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B2%5D%5Bdata%5D=BrandName&columns%5B2%5D%5Bname%5D=&columns%5B2%5D%5Bsearchable%5D=true&columns%5B2%5D%5Borderable%5D=true&columns%5B2%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B2%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B3%5D%5Bdata%5D=AAIA_BrandID&columns%5B3%5D%5Bname%5D=&columns%5B3%5D%5Bsearchable%5D=true&columns%5B3%5D%5Borderable%5D=true&columns%5B3%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B3%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B4%5D%5Bdata%5D=&columns%5B4%5D%5Bname%5D=&columns%5B4%5D%5Bsearchable%5D=true&columns%5B4%5D%5Borderable%5D=false&columns%5B4%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B4%5D%5Bsearch%5D%5Bregex%5D=false&order%5B0%5D%5Bcolumn%5D=3&order%5B0%5D%5Bdir%5D=asc&start=0&length=1000000&search%5Bvalue%5D=&search%5Bregex%5D=false&brandID='''+brand_id+'''",
              "method": "POST",
              "mode": "cors",
              "credentials": "include"
            }).then((response) => response.json()).then((data)=>datas=data)
        return datas
        ''')
        brands_info.append([brand_name,len(temp_lst['data'])])
        print(f"{brand_name}: {len(temp_lst['data'])}")
        for i in temp_lst['data']:
            all_data.append([i['BrandName'],brand_id,str(i['ProductID']),i['CategoryName'],i['PartNumber']])
    joblib.dump(brands_info, 'brands_info.sav')
    joblib.dump(all_data, 'all_brands_data.sav')


In [6]:
df_brand=pd.DataFrame(brands_info,columns=['Brand Name','Quantity of products'])
df_brand.to_csv('brands_info.csv',index=False)

In [7]:
chrome_options_helper = webdriver.ChromeOptions()
chrome_options_helper.add_argument("--incognito")
helper_driver = uc.Chrome(executable_path=ChromeDriverManager().install(),use_subprocess=True,options=chrome_options_helper)
helper_driver.delete_all_cookies()
helper_driver.get("https://apps.semadata.org/")
try:
    helper_driver.minimize_window()
except:
    pass
try:
    helper_driver.maximize_window()
except:
    pass
helper_driver.implicitly_wait(30)
helper_driver.find_element(By.XPATH,"//input[@placeholder='Username']").send_keys(username)
helper_driver.find_element(By.XPATH,"//input[@placeholder='Password']").send_keys(password)
helper_driver.find_element(By.ID,'login-button').click()
driver.get('https://apps.semadata.org/Receiver/GetProductDetail')
helper_driver.get('https://apps.semadata.org/Receiver/GetProductDetail')
helper_driver.set_script_timeout(1000)

In [11]:
# excel_data=[]

In [12]:
# unreachable=[]

In [29]:
for count,i in enumerate(all_data[15051:]):
    print(f'{count+15051}- {i[0]}')
    temp=[]
    description=None
    length=None
    width=None
    weight=None
    weight_UOM=None
    dimension_UOM=None
    height=None
    country_of_origin=None
    life_cycle_status_description=None
    refurbished_part=None
    package_level_GTIN=None
    warranty_time=None
    warranty_time_UOM=None
    taxable=None
    data=driver.execute_script('''
    var datas
    await fetch("https://apps.semadata.org/Receiver/GetProductDetail", {
      "headers": {
        "accept": "*/*",
        "accept-language": "en-GB,en-US;q=0.9,en;q=0.8",
        "content-type": "application/x-www-form-urlencoded; charset=UTF-8",
        "sec-fetch-dest": "empty",
        "sec-fetch-mode": "cors",
        "sec-fetch-site": "same-origin",
        "x-requested-with": "XMLHttpRequest"
      },
      "referrer": "https://apps.semadata.org/",
      "referrerPolicy": "strict-origin-when-cross-origin",
      "body": "productID='''+i[2]+'''&brandID='''+i[1]+'''",
      "method": "POST",
      "mode": "cors",
      "credentials": "include"
    }).then((response) => response.json()).then((data)=>datas=data)
    return datas
        ''')
    
    product_URL=f'https://apps.semadata.org/Receiver/GetProductPage?productID={i[2]}'
    html_text=helper_driver.execute_script('''
    var datas;
    await fetch("https://apps.semadata.org/Receiver/GetProductPage?productID='''+i[2]+'''", {
      "headers": {
        "accept": "application/json",
        "accept-language": "en-GB,en-US;q=0.9,en;q=0.8",
        "cache-control": "no-cache",
        "pragma": "no-cache",
        "sec-fetch-dest": "document",
        "sec-fetch-mode": "navigate",
        "sec-fetch-site": "same-origin",
        "sec-fetch-user": "?1",
        "upgrade-insecure-requests": "1"
      },
      "referrer": "https://apps.semadata.org/",
      "referrerPolicy": "strict-origin-when-cross-origin",
      "body": null,
      "method": "GET",
      "mode": "cors",
      "credentials": "include"
    }).then((response) => response.text()).then((data)=>datas=data)
    return datas
    ''')
    try:
        method1=True
        tree = html.fromstring(html_text)
    except:

            method1=False
            print('Trying with browser')
            helper_driver.get(f'https://apps.semadata.org/Receiver/GetProductPage?productID={i[2]}')
            try:
                helper_driver.refresh()
                element = WebDriverWait(driver, 60).until(
                    EC.presence_of_element_located((By.XPATH, "//div[@class='product-detail-main']//span[contains(text(),'Part Type')]/parent::*/text()"))
                )
            except:
                unreachable.append(i)
                print('Unreachable: ',len(unreachable))
                continue
            driver.execute_script('''location.reload(true)''')
            driver.implicitly_wait(30)

    try:
        price=float(data['ProductAttributes'][5]['ProductAttributes'][0]['ProductAttributes'][len(data['ProductAttributes'][5]['ProductAttributes'][0]['ProductAttributes'])-1]['ValueText'])
    except:
        continue
    currency=data['ProductAttributes'][5]['ProductAttributes'][0]['ProductAttributes'][len(data['ProductAttributes'][5]['ProductAttributes'][0]['ProductAttributes'])-2]['ValueText']
    brand_name=i[0]
    brand_id=i[1]
    product_id=i[2]
    category=i[3]
    part_number=i[4]
    if method1:
        description=assign(description,"//div[@class='product-detail-main']//span[contains(text(),'Description')]/parent::*/text()")
        length=assign(length,"(//span[contains(text(),'Length')]/parent::*/text())[2]")
        width=assign(width,"(//span[contains(text(),'Width')]/parent::*/text())[2]")
        weight=assign(weight,"(//span[contains(text(),'Weight')]/parent::*/text())[2]")
        weight_UOM=assign(weight_UOM,"(//span[contains(text(),'Weight UOM')]/parent::*/text())[2]")
        dimension_UOM=assign(dimension_UOM,"(//span[contains(text(),'Dimension UOM')]/parent::*/text())[2]")
        height=assign(height,"(//span[contains(text(),'Height')]/parent::*/text())[2]")
        country_of_origin=assign(country_of_origin,"(//span[contains(text(),'Country of Origin')]/parent::*/text())[2]")
        life_cycle_status_description=assign(life_cycle_status_description,"(//span[contains(text(),' Life Cycle Status Description')]/parent::*/text())[2]")
        refurbished_part=assign(refurbished_part,"(//span[contains(text(),'Refurbished ')]/parent::*/text())[2]")
        package_level_GTIN=str(assign(package_level_GTIN,"(//span[contains(text(),'Package Level GTIN')]/parent::*/text())[2]"))
        warranty_time=assign(warranty_time,"(//span[contains(text(),'Warranty Time')]/parent::*/text())[2]")
        warranty_time_UOM=assign(warranty_time_UOM,"(//span[contains(text(),'Warranty Time UOM')]/parent::*/text())[2]")
        taxable=assign(taxable,"(//span[contains(text(),'Taxable')]/parent::*/text())[2]")
    else:
        description=helper_driver.execute_script('''
        try{return document.evaluate("//div[@class='product-detail-main']//span[contains(text(),'Description')]/parent::*/text()", document, null, XPathResult.FIRST_ORDERED_NODE_TYPE, null).singleNodeValue.textContent.trim()}
        catch{return ' '}
        ''')
        length=helper_driver.execute_script('''
        try{return parseFloat(document.evaluate("(//span[contains(text(),'Length')]/parent::*/text())[2]", document, null, XPathResult.FIRST_ORDERED_NODE_TYPE, null).singleNodeValue.textContent.trim())}
        catch{return ' '}
        ''')
        width=helper_driver.execute_script('''
        try{return parseFloat(document.evaluate("(//span[contains(text(),'Width')]/parent::*/text())[2]", document, null, XPathResult.FIRST_ORDERED_NODE_TYPE, null).singleNodeValue.textContent.trim())}
        catch{return ' '}
        ''')
        weight=helper_driver.execute_script('''
        try{return parseFloat(document.evaluate("(//span[contains(text(),'Weight')]/parent::*/text())[2]", document, null, XPathResult.FIRST_ORDERED_NODE_TYPE, null).singleNodeValue.textContent.trim())}
        catch{return ' '}
        ''')
        weight_UOM=helper_driver.execute_script('''
        try{return document.evaluate("(//span[contains(text(),'Weight UOM')]/parent::*/text())[2]", document, null, XPathResult.FIRST_ORDERED_NODE_TYPE, null).singleNodeValue.textContent.trim()}
        catch{return ' '}
        ''')
        dimension_UOM=helper_driver.execute_script('''
        try{return document.evaluate("(//span[contains(text(),'Dimension UOM')]/parent::*/text())[2]", document, null, XPathResult.FIRST_ORDERED_NODE_TYPE, null).singleNodeValue.textContent.trim()}
        catch{return ' '}
        ''')
        height=helper_driver.execute_script('''
        try{return parseFloat(document.evaluate("(//span[contains(text(),'Height')]/parent::*/text())[2]", document, null, XPathResult.FIRST_ORDERED_NODE_TYPE, null).singleNodeValue.textContent.trim())}
        catch{return ' '}
        ''')
        country_of_origin=helper_driver.execute_script('''
        try{return document.evaluate("(//span[contains(text(),'Country of Origin')]/parent::*/text())[2]", document, null, XPathResult.FIRST_ORDERED_NODE_TYPE, null).singleNodeValue.textContent.trim()}
        catch{return ' '}
        ''')
        life_cycle_status_description=helper_driver.execute_script('''
        try{return document.evaluate("(//span[contains(text(),' Life Cycle Status Description')]/parent::*/text())[2]", document, null, XPathResult.FIRST_ORDERED_NODE_TYPE, null).singleNodeValue.textContent.trim()}
        catch{return ' '}
        ''')
        refurbished_part=helper_driver.execute_script('''
        try{return document.evaluate("(//span[contains(text(),'Refurbished ')]/parent::*/text())[2]", document, null, XPathResult.FIRST_ORDERED_NODE_TYPE, null).singleNodeValue.textContent.trim()}
        catch{return ' '}
        ''')
        package_level_GTIN=helper_driver.execute_script('''
        try{return document.evaluate("(//span[contains(text(),'Package Level GTIN')]/parent::*/text())[2]", document, null, XPathResult.FIRST_ORDERED_NODE_TYPE, null).singleNodeValue.textContent.trim()}
        catch{return ' '}
        ''')
        warranty_time=helper_driver.execute_script('''
        try{return document.evaluate("(//span[contains(text(),'Warranty Time')]/parent::*/text())[2]", document, null, XPathResult.FIRST_ORDERED_NODE_TYPE, null).singleNodeValue.textContent.trim()}
        catch{return ' '}
        ''')
        warranty_time_UOM=helper_driver.execute_script('''
        try{return document.evaluate("(//span[contains(text(),'Warranty Time UOM')]/parent::*/text())[2]", document, null, XPathResult.FIRST_ORDERED_NODE_TYPE, null).singleNodeValue.textContent.trim()}
        catch{return ' '}
        ''')
        taxable=helper_driver.execute_script('''
        try{return document.evaluate("(//span[contains(text(),'Taxable')]/parent::*/text())[2]", document, null, XPathResult.FIRST_ORDERED_NODE_TYPE, null).singleNodeValue.textContent.trim()}
        catch{return ' '}
        ''')

    try:
        data['ProductConfig']
    except:
        continue
    for j in data['ProductConfig']:
        vehicle_company=j['MakeName']
        vehicle_model=j['ModelName']
        quantity=j['Quantity']
        try:
            year=datetime.datetime.strptime(j['YearID'], "%Y").year
        except:
            year=j['YearID']
        if year==None:
            continue
        temp.append([product_id,brand_name,product_URL,vehicle_company,vehicle_model,quantity,year,brand_id,description,part_number,category,price,currency,length,width,height,weight,weight_UOM,dimension_UOM,country_of_origin,life_cycle_status_description,refurbished_part,package_level_GTIN,warranty_time,warranty_time_UOM,taxable])
    temp.sort(key=lambda x: x[6])
    for j in temp:
        excel_data.append(j)

15051- Black Horse Off Road
15052- Black Horse Off Road
15053- Black Horse Off Road
15054- Black Horse Off Road
15055- Black Horse Off Road
15056- Black Horse Off Road
15057- Black Horse Off Road
15058- Black Horse Off Road
15059- Black Horse Off Road
15060- Black Horse Off Road
15061- Black Horse Off Road
15062- Black Horse Off Road
15063- Black Horse Off Road
15064- Black Horse Off Road
15065- Black Horse Off Road
15066- Black Horse Off Road
15067- Black Horse Off Road
15068- Black Horse Off Road
15069- Black Horse Off Road
15070- Black Horse Off Road
15071- Black Horse Off Road
15072- Black Horse Off Road
15073- Black Horse Off Road
15074- Black Horse Off Road
15075- Black Horse Off Road
15076- Black Horse Off Road
15077- Black Horse Off Road
15078- Black Horse Off Road
15079- Black Horse Off Road
15080- Black Horse Off Road
15081- Black Horse Off Road
15082- Black Horse Off Road
15083- Black Horse Off Road
15084- Black Horse Off Road
15085- Black Horse Off Road
15086- Black Horse O

KeyboardInterrupt: 

In [30]:
df=pd.DataFrame(excel_data,columns=['ProductID','Brand Name','Product URL','Vehicle Company','Vehicle Model','Quantity','Year','BrandID','Description','Part Number','Category','Price','Currency','Length','Width','Height','Weight','Weight UOM','Dimension UOM','Country Of Origin','Life Cycle Status Description','Refurbished Part','Package Level GTIN','Warranty Time','Warranty Time UOM','Taxable'])
df.to_csv('final.csv',index=False)

In [32]:
# joblib.dump(excel_data, 'data_collected_so_far_2D_list.sav')

['data_collected_so_far_2D_list.sav']

In [ ]:
# description=helper_driver.execute_script('''
# try{return document.evaluate("//div[@class='product-detail-main']//span[contains(text(),'Description')]/parent::*/text()", document, null, XPathResult.FIRST_ORDERED_NODE_TYPE, null).singleNodeValue.textContent.trim()}
# catch{return ' '}
# ''')
# length=helper_driver.execute_script('''
# try{return parseFloat(document.evaluate("(//span[contains(text(),'Length')]/parent::*/text())[2]", document, null, XPathResult.FIRST_ORDERED_NODE_TYPE, null).singleNodeValue.textContent.trim())}
# catch{return ' '}
# ''')
# width=helper_driver.execute_script('''
# try{return parseFloat(document.evaluate("(//span[contains(text(),'Width')]/parent::*/text())[2]", document, null, XPathResult.FIRST_ORDERED_NODE_TYPE, null).singleNodeValue.textContent.trim())}
# catch{return ' '}
# ''')
# weight=helper_driver.execute_script('''
# try{return parseFloat(document.evaluate("(//span[contains(text(),'Weight')]/parent::*/text())[2]", document, null, XPathResult.FIRST_ORDERED_NODE_TYPE, null).singleNodeValue.textContent.trim())}
# catch{return ' '}
# ''')
# weight_UOM=helper_driver.execute_script('''
# try{return document.evaluate("(//span[contains(text(),'Weight UOM')]/parent::*/text())[2]", document, null, XPathResult.FIRST_ORDERED_NODE_TYPE, null).singleNodeValue.textContent.trim()}
# catch{return ' '}
# ''')
# dimension_UOM=helper_driver.execute_script('''
# try{return document.evaluate("(//span[contains(text(),'Dimension UOM')]/parent::*/text())[2]", document, null, XPathResult.FIRST_ORDERED_NODE_TYPE, null).singleNodeValue.textContent.trim()}
# catch{return ' '}
# ''')
# height=helper_driver.execute_script('''
# try{return parseFloat(document.evaluate("(//span[contains(text(),'Height')]/parent::*/text())[2]", document, null, XPathResult.FIRST_ORDERED_NODE_TYPE, null).singleNodeValue.textContent.trim())}
# catch{return ' '}
# ''')
# country_of_origin=helper_driver.execute_script('''
# try{return document.evaluate("(//span[contains(text(),'Country of Origin')]/parent::*/text())[2]", document, null, XPathResult.FIRST_ORDERED_NODE_TYPE, null).singleNodeValue.textContent.trim()}
# catch{return ' '}
# ''')
# life_cycle_status_description=helper_driver.execute_script('''
# try{return document.evaluate("(//span[contains(text(),' Life Cycle Status Description')]/parent::*/text())[2]", document, null, XPathResult.FIRST_ORDERED_NODE_TYPE, null).singleNodeValue.textContent.trim()}
# catch{return ' '}
# ''')
# refurbished_part=helper_driver.execute_script('''
# try{return document.evaluate("(//span[contains(text(),'Refurbished ')]/parent::*/text())[2]", document, null, XPathResult.FIRST_ORDERED_NODE_TYPE, null).singleNodeValue.textContent.trim()}
# catch{return ' '}
# ''')
# package_level_GTIN=helper_driver.execute_script('''
# try{return document.evaluate("(//span[contains(text(),'Package Level GTIN')]/parent::*/text())[2]", document, null, XPathResult.FIRST_ORDERED_NODE_TYPE, null).singleNodeValue.textContent.trim()}
# catch{return ' '}
# ''')
# warranty_time=helper_driver.execute_script('''
# try{return document.evaluate("(//span[contains(text(),'Warranty Time')]/parent::*/text())[2]", document, null, XPathResult.FIRST_ORDERED_NODE_TYPE, null).singleNodeValue.textContent.trim()}
# catch{return ' '}
# ''')
# warranty_time_UOM=helper_driver.execute_script('''
# try{return document.evaluate("(//span[contains(text(),'Warranty Time UOM')]/parent::*/text())[2]", document, null, XPathResult.FIRST_ORDERED_NODE_TYPE, null).singleNodeValue.textContent.trim()}
# catch{return ' '}
# ''')
# taxable=helper_driver.execute_script('''
# try{return document.evaluate("(//span[contains(text(),'Taxable')]/parent::*/text())[2]", document, null, XPathResult.FIRST_ORDERED_NODE_TYPE, null).singleNodeValue.textContent.trim()}
# catch{return ' '}
# ''')


In [14]:
helper_driver.refresh()